In [ ]:
# For recent files

# import pandas as pd

# # Read the most recent dataset
# import glob
# import os

# # Find the most recent parquet file
# parquet_files = glob.glob('data/datasets/mbpp_dataset_*.parquet')
# if parquet_files:
#     latest_file = max(parquet_files, key=os.path.getmtime)
#     print(f"Loading: {latest_file}")
#     df = pd.read_parquet(latest_file)
# else:
#     print("No parquet files found")
#     df = None

# if df is not None:
#     print(f"Dataset shape: {df.shape}")
#     print(f"Columns: {df.columns.tolist()}")
    
#     # Show basic statistics
#     if 'complexity_score' in df.columns:
#         print(f"\nComplexity score statistics:")
#         print(df['complexity_score'].describe())
        
#         # Show sample data
#         print(f"\nSample data:")
#         display_cols = ['task_id', 'test_passed', 'complexity_score']
#         print(df[display_cols].head())
#     else:
#         print("\nComplexity score column not found!")
#         print("Available columns:", df.columns.tolist())

No parquet files found


In [ ]:
# For handpicked files

In [2]:
# Force reload the updated DatasetManager module to clear cache
import importlib
import sys

# Remove the module from cache if it exists
if 'phase1_dataset_building.dataset_manager' in sys.modules:
    del sys.modules['phase1_dataset_building.dataset_manager']
if 'phase1_dataset_building' in sys.modules:
    del sys.modules['phase1_dataset_building']

# Now import the updated DatasetManager
from phase1_dataset_building.dataset_manager import DatasetManager

try:
    # Initialize the dataset manager
    dataset_manager = DatasetManager()
    
    # Load the full MBPP dataset
    print("Loading MBPP dataset using updated DatasetManager (after module reload)...")
    dataset_manager.load_dataset()
    
    print(f"✅ Successfully loaded MBPP dataset")
    print(f"Total records: {dataset_manager.get_size()}")
    print(f"Expected: 974 records")
    
    if dataset_manager.get_size() == 974:
        print("✅ All 974 records loaded successfully!")
    else:
        print(f"❌ Expected 974 records, but got {dataset_manager.get_size()}")
        print("The module may still be cached. Try restarting the kernel.")
    
    # Show some basic info about the dataset
    print(f"\nDataset structure:")
    if dataset_manager.is_loaded():
        sample_record = dataset_manager.get_record(0)
        print(f"Keys in each record: {list(sample_record.keys())}")
        print(f"First task_id: {sample_record.get('task_id', 'unknown')}")
        print(f"Text: {sample_record.get('text', '')[:100]}...")

except Exception as e:
    print(f"❌ Error loading MBPP dataset: {e}")
    import traceback
    traceback.print_exc()

ModuleNotFoundError: No module named 'phase1_dataset_building'

In [31]:
# Alternative: Test with direct dataset loading to verify our fix
from datasets import load_dataset

# Test the exact same call that DatasetManager should now make
print("Testing direct call to load_dataset('Muennighoff/mbpp', 'full')...")
dataset = load_dataset("Muennighoff/mbpp", "full")
print(f"Direct load result: {len(dataset['test'])} records")

# Now test DatasetManager
from phase1_dataset_building.dataset_manager import DatasetManager
dm = DatasetManager()
dm.load_dataset()
print(f"DatasetManager result: {dm.get_size()} records")

Testing direct call to load_dataset('Muennighoff/mbpp', 'full')...
Direct load result: 974 records
DatasetManager result: 974 records


In [37]:
# Let's examine what code is being generated and the prompts
import pandas as pd
import glob
import os
from phase1_dataset_building.dataset_manager import DatasetManager, PromptTemplateBuilder

# Find the most recent parquet file
parquet_files = glob.glob('data/datasets/mbpp_dataset_*.parquet')
if parquet_files:
    latest_file = max(parquet_files, key=os.path.getmtime)
    df = pd.read_parquet(latest_file)
    
    # Load dataset manager to see original tasks
    dm = DatasetManager()
    dm.load_dataset()
    template_builder = PromptTemplateBuilder()
    
    print("=== ANALYSIS OF GENERATED CODE ===\n")
    for idx, row in df.iterrows():
        task_id = row['task_id']
        print(f"Task ID: {task_id}")
        print(f"Test Passed: {row['test_passed']}")
        
        # Get original record
        record = dm.get_record(task_id - 1)  # task_id is 1-indexed
        print(f"\nOriginal Problem: {record['text']}")
        print(f"Test Cases: {record['test_list']}")  # Show first 2
        
        # Show the prompt that would be generated
        prompt = template_builder.build_prompt(record)
        print(f"\nGenerated Prompt:\n{prompt}")
        
        print(f"\nGenerated Code:\n{row['generated_code']}")
        print("=" * 80)
        print()

=== ANALYSIS OF GENERATED CODE ===

Task ID: 1
Test Passed: False

Original Problem: Write a function to find the minimum cost path to reach (m, n) from (0, 0) for the given cost matrix cost[][] and a position (m, n) in cost[][].
Test Cases: ['assert min_cost([[1, 2, 3], [4, 8, 2], [1, 5, 3]], 2, 2) == 8', 'assert min_cost([[2, 3, 4], [5, 9, 3], [2, 6, 4]], 2, 2) == 12', 'assert min_cost([[3, 4, 5], [6, 10, 4], [3, 7, 5]], 2, 2) == 16']

Generated Prompt:
Write a function to find the minimum cost path to reach (m, n) from (0, 0) for the given cost matrix cost[][] and a position (m, n) in cost[][].

assert min_cost([[1, 2, 3], [4, 8, 2], [1, 5, 3]], 2, 2) == 8
assert min_cost([[2, 3, 4], [5, 9, 3], [2, 6, 4]], 2, 2) == 12
assert min_cost([[3, 4, 5], [6, 10, 4], [3, 7, 5]], 2, 2) == 16

# Your code here

Generated Code:

def min_cost(cost, m, n):
    # Write your code here
    dp = [[0 for i in range(n+1)] for j in range(m+1)]
    for i in range(1, m+1):
        for j in range(1, n+1):
 

In [34]:
# Let's test the prompt generation and model output directly
from phase1_dataset_building.dataset_manager import DatasetManager, PromptTemplateBuilder
from common.models import ModelManager
from common.config import ModelConfiguration

# Initialize components
dm = DatasetManager()
dm.load_dataset()
template_builder = PromptTemplateBuilder()

# Get a specific task that should be easy (task 11 from your dataset)
record = dm.get_record(10)  # task_id 11 is at index 10
print(f"Task ID: {record['task_id']}")
print(f"Problem: {record['text']}")
print(f"Test cases: {record['test_list']}")
print(f"Reference solution: {record['code']}")

# Build the prompt
prompt = template_builder.build_prompt(record)
print("\n" + "="*60)
print("GENERATED PROMPT:")
print("="*60)
print(prompt)
print("="*60)

# Check if the prompt ends with the code initiator
print(f"\nPrompt ends with '{template_builder.CODE_INITIATOR}': {prompt.endswith(template_builder.CODE_INITIATOR)}")
print(f"Prompt length: {len(prompt)} characters")

Task ID: 11
Problem: Write a python function to remove first and last occurrence of a given character from the string.
Test cases: ['assert remove_Occ("hello","l") == "heo"', 'assert remove_Occ("abcda","a") == "bcd"', 'assert remove_Occ("PHP","P") == "H"']
Reference solution: def remove_Occ(s,ch): 
    for i in range(len(s)): 
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    for i in range(len(s) - 1,-1,-1):  
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    return s 

GENERATED PROMPT:
Write a python function to remove first and last occurrence of a given character from the string.

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"

# Your code here

Prompt ends with '# Your code here': True
Prompt length: 232 characters


In [35]:
# Test actual model generation
from common.config import ModelConfiguration

# Create model config
model_config = ModelConfiguration(
    model_name="google/gemma-2-2b",
    temperature=0.0,  # Deterministic
    do_sample=False,
    max_new_tokens=200
)

# Initialize model manager
model_manager = ModelManager(model_config)
print("Loading model...")
model_manager.load_model()

# Test generation with our prompt
print("\nGenerating code...")
generated_code = model_manager.generate(prompt, max_new_tokens=200, stream=True)

print("\n" + "="*60)
print("GENERATED CODE (raw):")
print("="*60)
print(repr(generated_code))
print("="*60)

# Check if the generated code is actually Python code
print(f"\nGenerated code length: {len(generated_code)} characters")
print(f"Contains 'def': {'def' in generated_code}")
print(f"First 100 chars: {generated_code[:100]}")

# Clean up
model_manager.cleanup()

Loading model...


Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.20s/it]
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Generating code...
Write a python function to remove first and last occurrence of a given character from the string.

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"

# Your code here
def remove_Occ(string,char):
    string = string.replace(char,"")
    return string


GENERATED CODE (raw):
'\ndef remove_Occ(string,char):\n    string = string.replace(char,"")\n    return string\n'

Generated code length: 85 characters
Contains 'def': True
First 100 chars: 
def remove_Occ(string,char):
    string = string.replace(char,"")
    return string



In [36]:
# Check the latest generated parquet file and display first 5 records
import pandas as pd
import glob
import os

# Find the most recent parquet file
parquet_files = glob.glob('data/datasets/mbpp_dataset_*.parquet')
if parquet_files:
    latest_file = max(parquet_files, key=os.path.getmtime)
    print(f"Latest parquet file: {latest_file}")
    print(f"Modified: {pd.Timestamp.fromtimestamp(os.path.getmtime(latest_file))}")
    
    # Load the dataset
    df = pd.read_parquet(latest_file)
    print(f"\nDataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    
    # Display the first 5 records as a table
    print("\nFirst 5 records:")
    display(df.head())
    
    # Show value counts for test_passed
    print("\nTest passed distribution:")
    print(df['test_passed'].value_counts())
    
    # Show complexity score distribution if available
    if 'complexity_score' in df.columns:
        print("\nComplexity score distribution:")
        print(df['complexity_score'].value_counts().sort_index())
else:
    print("No parquet files found in data/datasets/")

Latest parquet file: data/datasets/mbpp_dataset_20250528_152327.parquet
Modified: 2025-05-28 15:23:27.750401

Dataset shape: (4, 4)
Columns: ['task_id', 'generated_code', 'test_passed', 'complexity_score']

First 5 records:


,task_id,generated_code,test_passed,complexity_score
0,1,"\ndef min_cost(cost, m, n):\n # Write your code here\n dp = [[0 for i in range(n+1)] for j...",False,7
1,2,"\ndef similar_elements(list1, list2):\n # Write your code here\n return list(set(list1) & ...",False,1
2,3,"\ndef is_not_prime(n):\n if n == 1:\n return False\n for i in range(2, n):\n ...",False,3
3,4,"\ndef heap_queue_largest(arr,n):\n # Write your code here\n heap = []\n for i in range(...",False,1



Test passed distribution:
test_passed
False    4
Name: count, dtype: int64

Complexity score distribution:
complexity_score
1    2
3    1
7    1
Name: count, dtype: int64
